todo: 

* motor-ansteurerung so bauen wir im ursprünglichem experiment
* mehr schreiben was die step funktion macht

(ignore this cell)

# Neurorobotik with PyNN and PyBullet

In this Notebook we will build and execute a neuro-robotics experiment. Für the neural simulation we will use [PyNN](https://neuralensemble.org/PyNN/) and [Nest](https://www.nest-initiative.org/?page=Software). For the simulation of the physic we use [PyBullet](https://github.com/bulletphysics/bullet3/tree/master/examples/pybullet). 

This experiment is a re-creation of the [braitenberg-husky-experiment](https://bitbucket.org/hbpneurorobotics/experiments/src/development/braitenberg_husky/) in the neurorobotics plattform of the human brain project. 

We start by importing all modules we will use in this notebook. Make sure to install only python**3**-versions of all dependencies. 

You can not just install these via pip:

- _pybullet_ needs to be installed from source to get numpy-support. Otherwise it will be slower, and getCameraImage doesn't return a numpy-array. 
- _nest_ can only be installed from source. We must use version 2.16, because the current master is not yet compatible with _pynn_. Also _nest_ is compiled with _libnreuosim_, which needs a workaround until [the PR](https://github.com/nest/nest-simulator/pull/1235) is merged. I'm not 100% certain if _libneurosim_ is even required for this project, but _nest_ gives a warning if it's missing, so we will install it. 
- there is a warning "UserWarning: Unable to install NEST extensions. Certain models may not be available", which doesn't seem to affect this project. Please ignore it. 

In [1]:
import time
import numpy as np

import pybullet as p
import pybullet_data

import pyNN.nest as brain_sim
from pyNN.utility import get_simulator, normalized_filename

from matplotlib import pyplot as plt
from matplotlib import animation
from pyNN.utility.plotting import Figure, Panel
from quantities import mV

if not p.isNumpyEnabled():
    raise Exception("pybullet must be compiled with numpy-support for this projet")

/opt/conda/lib/python3.7/site-packages/pyNN/nest/__init__.py:55: UserWarning: Unable to install NEST extensions. Certain models may not be available.
Further details: DynamicModuleManagementError in Install: Module 'pynn_extensions' could not be opened.
The dynamic loader returned the following error: 'file not found'.

Please check LD_LIBRARY_PATH (OSX: DYLD_LIBRARY_PATH)!
  warnings.warn("Unable to install NEST extensions. Certain models may not be available.\nFurther details: {}".format(err))


### brain sim mit pynn

first we define the population in pynn. This code is copied from [braitenberg.py on hbpneurorobotics](https://bitbucket.org/hbpneurorobotics/models/src/development/brains/braitenberg.py)

In [2]:
def get_pop():
    sim = brain_sim
    SENSORPARAMS = {'v_rest': -60.5,
                    'cm': 0.025,
                    'tau_m': 10.,
                    'tau_refrac': 10.0,
                    'tau_syn_E': 2.5,
                    'tau_syn_I': 2.5,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_thresh': -60.0,
                    'v_reset': -60.5}

    GO_ON_PARAMS = {'v_rest': -60.5,
                    'cm': 0.025,
                    'tau_m': 10.0,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_reset': -61.6,
                    'v_thresh': -60.51,
                    'tau_refrac': 10.0,
                    'tau_syn_E': 2.5,
                    'tau_syn_I': 2.5}

    population = sim.Population(8, sim.IF_cond_alpha())
    population[0:5].set(**SENSORPARAMS)
    population[5:6].set(**GO_ON_PARAMS)
    population[6:8].set(**SENSORPARAMS)

    syn_params = {'U': 1.0, 'tau_rec': 1.0, 'tau_facil': 1.0}

    # Synaptic weights
    WEIGHT_RED_TO_ACTOR = 1.5e-4
    WEIGHT_RED_TO_GO_ON = 1.2e-3  # or -1.2e-3?
    WEIGHT_GREEN_BLUE_TO_ACTOR = 1.05e-4
    WEIGHT_GO_ON_TO_RIGHT_ACTOR = 1.4e-4
    DELAY = 1

    # Connect neurons
    CIRCUIT = population

    SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_RED_TO_ACTOR),
                                    delay=DELAY, **syn_params)
    sim.Projection(presynaptic_population=CIRCUIT[2:3],
                   postsynaptic_population=CIRCUIT[7:8],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')
    sim.Projection(presynaptic_population=CIRCUIT[3:4],
                   postsynaptic_population=CIRCUIT[6:7],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')


    SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_RED_TO_GO_ON),
                                    delay=DELAY, **syn_params)
    sim.Projection(presynaptic_population=CIRCUIT[0:2],
                   postsynaptic_population=CIRCUIT[4:5],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='inhibitory')
    sim.Projection(presynaptic_population=CIRCUIT[0:2],
                   postsynaptic_population=CIRCUIT[5:6],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='inhibitory')

    SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_GREEN_BLUE_TO_ACTOR),
                                    delay=DELAY, **syn_params)
    sim.Projection(presynaptic_population=CIRCUIT[4:5],
                   postsynaptic_population=CIRCUIT[7:8],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')

    SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_GO_ON_TO_RIGHT_ACTOR),
                                    delay=DELAY, **syn_params)
    sim.Projection(presynaptic_population=CIRCUIT[5:6],
                   postsynaptic_population=CIRCUIT[7:8],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')

    return population    




next we set up the input for the population. For this we create two poisson spike generators and hook them up to the population according to original project. The values for the projections come from in the first 6 lines of [this file](https://bitbucket.org/hbpneurorobotics/experiments/src/development/braitenberg_husky/eye_sensor_transmit.py) and the parameters for the new neurons come are the [default values for PoissonSpikeGenerators](https://bitbucket.org/hbpneurorobotics/cle/src/development/hbp_nrp_cle/hbp_nrp_cle/brainsim/pynn/devices/__PyNNPoissonSpikeGenerator.py)

In [3]:
class SpikeGenerator(object):
    """the class handles the input for the brain"""
    min_rate = 0.0005
    def __init__(self, population, interval=20.0, count_red_left=1/3, count_red_right=1/3, count_non_red=1/3):
        params_source=  {'start': 0.0, 'duration': float("inf"), 'rate': 0.0}
        params_connec= {'weight':0.00015, 'receptor_type':'excitatory','delay':0.1}
        self.ssp_red_left_eye = brain_sim.create(brain_sim.SpikeSourcePoisson,  params_source)
        self.ssp_red_right_eye = brain_sim.create(brain_sim.SpikeSourcePoisson,  params_source)
        self.ssp_green_blue_eye = brain_sim.create(brain_sim.SpikeSourcePoisson,  params_source)
        brain_sim.connect(self.ssp_red_left_eye, population[slice(0, 3, 2)], **params_connec)
        brain_sim.connect(self.ssp_red_right_eye, population[slice(1, 4, 2)], **params_connec)
        brain_sim.connect(self.ssp_green_blue_eye, population[4], **params_connec)
    
    def set_rates_from_image(self, img):
        ratio = self._get_red_ration_from_image(img)
        self._set_rates_from_ratios(*ratio)
    
    @staticmethod
    def _get_red_ration_from_image(img):
        """detect left red and right red like nrp
        pixelformat is guessed, but it seems to work.
        this functions tries to reproduce https://developer.humanbrainproject.eu/docs/projects/hbp-nrp-cle/1.1.5/codedoc/hbp_nrp_cle.tf_framework.html#hbp_nrp_cle.tf_framework.tf_lib.detect_red"""
        w = img[0]  # width of the image, in pixels
        h = img[1]  # height of the image, in pixels
        rgb_buffer = img[2]  # color data RGB
        # dep_buffer = img_arr[3]  # depth data
        count_red_right = 0
        count_red_left = 0
        count_non_red = 0
        for y in range(h):
            for x in range(w):
                # numpy image
                r, g, b, a = (rgb_buffer[y][x][0], rgb_buffer[y][x][1], rgb_buffer[y][x][2], rgb_buffer[y][x][3])
                if r > g and r > b:
                    if x > w / 2:
                        count_red_right = count_red_right + 1
                    else:
                        count_red_left = count_red_left + 1
                else:
                    count_non_red = count_non_red + 1

        total = count_red_left + count_red_right + count_non_red
        return count_red_left/total, count_red_right/total, count_non_red/total

    
    def _set_rates_from_ratios(self, count_red_left, count_red_right, count_non_red):
        if count_red_left <= self.min_rate:
            count_red_left = self.min_rate
        if count_red_right <= self.min_rate:
            count_red_right = self.min_rate
        if count_non_red <= self.min_rate:
            count_non_red = self.min_rate
        try:
            self.ssp_red_left_eye.set(rate=2*2000.0 * count_red_left)
            self.ssp_red_right_eye.set(rate=2*2000.0 * count_red_right)
            self.ssp_green_blue_eye.set(rate=75.0 * count_non_red)
        except StopIteration:
            pass

the next class handles the output of the brain. It connects two leaky-integrator neurons tp the output-neurons to convert their spikes into voltages, which we will measure. 
        
        
The parameters for these Neurons come from [defaults](https://developer.humanbrainproject.eu/docs/projects/hbp-nrp-cle/1.3.8/modules/hbp_nrp_cle/brainsim/pynn/devices/__PyNNLeakyIntegratorTypes.html), again. The connection to the population come from [line 10 and 11 of this file from the original experiment](https://bitbucket.org/hbpneurorobotics/experiments/src/d6e01275e51e0db8fc341bfe5873cb805d4c5c44/braitenberg_husky/braitenberg_husky_linear_twist.py#lines-10)


In [4]:

class OutputGenerator(object):
    def __init__(self, population):
        LI_create = {'v_thresh': float('inf'),'cm': 1.0,'tau_m': 10.0,'tau_syn_E': 2.,
            'tau_syn_I': 2.,'v_rest': 0.0,'v_reset': 0.0,'tau_refrac': 0.1,}
        LI_connect = {'weight':0.00015, 'receptor_type':'excitatory','delay':0.1}
        self.left = brain_sim.create(brain_sim.IF_curr_alpha, LI_create)
        self.right = brain_sim.create(brain_sim.IF_curr_alpha, LI_create)
        brain_sim.connect(population[6], self.left, **LI_connect)
        brain_sim.connect(population[7],self.right,  **LI_connect)
        brain_sim.initialize(self.left, v=self.left.get('v_rest'))
        brain_sim.initialize(self.right, v=self.right.get('v_rest'))
        self.left.record('v')
        self.right.record('v')
    
    def get_current_voltage(self):
        return self._get_current_voltage(self.left), self._get_current_voltage(self.right)
    def _get_current_voltage(self, cell):
        return cell.get_data('v', clear=True).segments[0].filter(name="v")[0][-1,0]

### physics with PyBullet

now lets set up the physical world for the experiment. 

We set up a simple world. It has a generic plane for a floor. And there are two floating rectangles which switch color every couple of seconds. 

In [5]:
#setup physics sim with pybullet

class World(object):
    """this class handles the world sans robot
    the world is just a plane, with two rectangles, which switch colors every 5000 steps"""
    green = [0, 1, 0, 1]
    blue = [0, 0, 1, 1]
    red = [1, 0, 0, 1]
    count_steps = 0
    count_switch = 0
    age = 0.0
    last_switch = float('-inf')

    def __init__(self):
        p.setAdditionalSearchPath(pybullet_data.getDataPath())  # used by loadURDF
        p.setGravity(0, 0, -10)
        p.loadURDF("plane.urdf")
        plane_id = p.createCollisionShape(p.GEOM_BOX, halfExtents=[1, 0.25, 1])
        plane_vis_id = p.createVisualShape(p.GEOM_BOX, halfExtents=[1, 0.25, 1], rgbaColor=self.blue)
        self.planeA = p.createMultiBody(baseCollisionShapeIndex=plane_id, basePosition=[0, 5, 2],
                                        baseVisualShapeIndex=plane_vis_id)
        self.planeB = p.createMultiBody(baseCollisionShapeIndex=plane_id, basePosition=[0, -5, 2],
                                        baseVisualShapeIndex=plane_vis_id)
        p.setRealTimeSimulation(0)

    def do_step(self):
        # PyBullet doesn't track this according to 
        # [this forum post](https://pybullet.org/Bullet/phpBB3/viewtopic.php?t=12438), 
        # so we need to track it ourselves
        self.age = self.age + p.getPhysicsEngineParameters()['fixedTimeStep']
        self.count_steps = self.count_steps + 1
        if self.age - self.last_switch > 10:
            self.last_switch = self.age
            self.count_switch = self.count_switch + 1
            if self.count_switch % 2 == 0:
                p.changeVisualShape(self.planeA, -1, rgbaColor=self.green)
                p.changeVisualShape(self.planeB, -1, rgbaColor=self.red)
            else:
                p.changeVisualShape(self.planeA, -1, rgbaColor=self.red)
                p.changeVisualShape(self.planeB, -1, rgbaColor=self.green)
        p.stepSimulation()



Now we create a body for the robot. We use [husky](https://clearpathrobotics.com/husky-unmanned-ground-vehicle-robot/), which is a default model both for virtual and real-world robotics experiments. 

To this body we add a virtual camera, which will feed the brain with input, and drivers for the wheels, which will be controlled by the brain later.

In [6]:
class Husky(object):
    maxForce = 100
    targetVelocity = 10

    class Camera(object):
        init_camera_vector = (1, 0, 0)
        init_up_vector = (0, 0, 1)

        def __init__(self, pixel_width=80, pixel_height=80, near_plane=0.1, far_plane=100, fov=100):
            self.pixel_width = pixel_width
            self.pixel_height = pixel_height
            self.projection_mat = p.computeProjectionMatrixFOV(
                fov, 
                pixel_width / pixel_height, 
                near_plane, 
                far_plane)
        
        def get_image(self, cam_pos, cam_orientation):
            cam_rot = np.array(p.getMatrixFromQuaternion(cam_orientation)).reshape(3, 3)
            cam_direction = cam_rot.dot(self.init_camera_vector)
            cam_up_vec = cam_rot.dot(self.init_up_vector)
            # note: if cam is exactly on joint, getCameraImage freezes sometimes, so we move it a little
            # see https://github.com/bulletphysics/bullet3/issues/2297
            cam_pos = cam_pos + (cam_direction * 0.5)
            cam_target = cam_pos + (cam_direction * 2.0)
            view_mat = p.computeViewMatrix(cam_pos, cam_target, cam_up_vec)

            return p.getCameraImage(self.pixel_width,
                                    self.pixel_height,
                                    viewMatrix=view_mat,
                                    projectionMatrix=self.projection_mat,
                                    flags=p.ER_NO_SEGMENTATION_MASK)


    def __init__(self):
        self.camera = Husky.Camera()
        self.husky_model = p.loadURDF("husky/husky.urdf",
                                      basePosition=[0, 0, 1],
                                      baseOrientation=p.getQuaternionFromEuler([0, 0, 0]))

        for joint in range(p.getNumJoints(self.husky_model)):
            if p.getJointInfo(self.husky_model, joint)[1] == b'user_rail':
                self.camera_joint = joint
            if p.getJointInfo(self.husky_model, joint)[1] == b'front_left_wheel':
                self.front_left_wheel = joint
            if p.getJointInfo(self.husky_model, joint)[1] == b'front_right_wheel':
                self.front_right_wheel = joint
            if p.getJointInfo(self.husky_model, joint)[1] == b'rear_left_wheel':
                self.rear_left_wheel = joint
            if p.getJointInfo(self.husky_model, joint)[1] == b'rear_right_wheel':
                self.rear_right_wheel = joint
    
    def get_camera_image(self):
        joint_pos, joint_orn, _, _, _, _ = p.getLinkState(self.husky_model, self.camera_joint,
                                                          computeForwardKinematics=True)
        return self.camera.get_image(joint_pos, joint_orn)
            
    def update_control(self, command_left, command_right):
        p.setJointMotorControl2(self.husky_model, self.front_left_wheel, p.VELOCITY_CONTROL,
                                targetVelocity=command_left * self.targetVelocity,
                                force=self.maxForce)
        p.setJointMotorControl2(self.husky_model, self.rear_left_wheel, p.VELOCITY_CONTROL,
                                targetVelocity=command_left * self.targetVelocity,
                                force=self.maxForce)
        p.setJointMotorControl2(self.husky_model, self.front_right_wheel, p.VELOCITY_CONTROL,
                                targetVelocity=command_right * self.targetVelocity,
                                force=self.maxForce)
        p.setJointMotorControl2(self.husky_model, self.rear_right_wheel, p.VELOCITY_CONTROL,
                                targetVelocity=command_right * self.targetVelocity,
                                force=self.maxForce)


### run experiment

now that we have defined everything we need, lets boot up experiment.

We will work with synchronized timesteps in both the brain simulation and the physics simulation. The default in bullet is [1ms](https://github.com/bulletphysics/bullet3/blob/d220101c5aad92cf7013710b0ed011395cabbb23/examples/pybullet/pybullet.c#L3001). We will be much slower than the defaults. On one hand we only need to update our inputs only a couple of times per seconds. On the other hand the rendering of the robot-images and the updating of the motor-input takes up a significant amount of processing. 

if the timesteps are too short, the simulation is very slow. If the timesteps are too long, the robot won't notice the red-area until it's too late. 

In [7]:
timestep_ms = 7

#### initialize the brain simulation 

<small>Note: the _timestep_ variable in brain_sim.setup() referes to the internal timesteps, and not to the external kind</small>

In [8]:
brain_sim.setup(timestep=0.1,min_delay=0.1,max_delay=4.0)
population=get_pop()
brain_sim.initialize(population, v=population.get('v_rest'))
spike_generator = SpikeGenerator(population)
output_generator = OutputGenerator(population)

#### initialize the physic simulation

In [9]:
physicsClient = p.connect(p.DIRECT)  
p.setTimeStep(timestep_ms/1000)
physic_sim = World()
robot = Husky()

#### run experiment

and now lets to a single step of the simulation. We wrap it in a function, so we can repeat the step at later points

In [10]:
def step():
    physic_sim.do_step()
    robot_img = robot.get_camera_image()
    spike_generator.set_rates_from_image(robot_img)
    brain_sim.run(timestep_ms)
    voltage = output_generator.get_current_voltage()
    # todo: get better numbers from experiment
    commands = np.multiply(np.subtract(voltage, 0.00015),4000)
    robot.update_control(commands[0].item(), commands[1].item() )
    return voltage, robot_img
    
step()
print("done")

done


that was fun, wasn't it? 

So lets do that again 100 times more

In [11]:
# %timeit  -n20 -r5 step()

Now lets take a look on what's actually going on in the experiment. 

We will now set up 3 plots:

* the first shows what the robot sees
* the second shows how the world outside the robot looks like
* the third shows the output-voltage of the brain

In [12]:
%matplotlib widget


# get some initial values by doing a single iteration
voltages,robot_img = step()


# the data looks like the, but it will be empty, because we clear it on every iteration
world_view_mat = p.computeViewMatrix([-5, 5, 5], [1, 0, -3], [0, 0, 1])
world_projection_mat = p.computeProjectionMatrixFOV(fov=80, aspect=1.0/1.0, nearVal=0.1, farVal=100)
world_img = p.getCameraImage(100,100, viewMatrix = world_view_mat, projectionMatrix=world_projection_mat)

# setup plots for physic
fig = plt.figure()
plt.subplot(221)
robot_plot = plt.imshow(robot_img[2], interpolation='none')
plt.subplot(222)
world_plot = plt.imshow(world_img[2], interpolation='none')

# setup plots for brain
brain_plot = plt.subplot(212)
brain_plot_length = 40
all_left = np.zeros(brain_plot_length)
all_right = np.zeros(brain_plot_length)
plt_left = plt.plot(all_left)
plt_right = plt.plot(all_right)
plt.ylabel("Voltage (mV)")
plt.ylim(-0.0001, 0.0009)
plt.xlim(0.0, brain_plot_length)


def animate(i):    
    voltages,robot_img = step()
    
    update_voltage_plot(voltages)
    update_physic_plot(robot_img)

def update_physic_plot(robot_img):
    robot_plot.set_data(robot_img[2])
    world_img = p.getCameraImage(100,100, viewMatrix = world_view_mat, projectionMatrix=world_projection_mat)
    world_plot.set_data(world_img[2])

def update_voltage_plot(voltages):
    global all_left, all_right
    all_left = np.append(all_left, voltages[0])
    all_right = np.append(all_right, voltages[1])
    all_left = np.delete(all_left,0)
    all_right = np.delete(all_right,0)
    plt_left[0].set_data(np.arange(brain_plot_length),all_left)
    plt_right[0].set_data(np.arange(brain_plot_length),all_right)


plt.show()
anim1 = animation.FuncAnimation(fig, animate, repeat = True, interval=10, frames=50)


FigureCanvasNbAgg()

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


to start the animation, you may have run the last cell manually once (ctrl+enter)

lets see if the timing in the brain simulation is still in sync with the time in the physic simulation

In [13]:
print(brain_sim.get_current_time()/1000)
print(physic_sim.age)

0.014
0.014
